In [52]:
#Importing the required libraries
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers
from sklearn.metrics import classification_report, accuracy_score

In [24]:
# getting the current working directory, and initializing image and label list
base_path = 'Dataset/Animals-10/raw-img'
images = []
labels = []

In [27]:
def load_images_from_subfolders(base_folder, target_size=(128, 128)):
    data = []
    labels = []
    for subdir, _, files in os.walk(base_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                img_path = os.path.join(subdir, file)
                img = Image.open(img_path)
                img = img.resize(target_size)
                img_array = np.array(img)
                if img_array.shape == (target_size[0], target_size[1], 3): 
                    data.append(img_array)
                    label = os.path.basename(subdir)
                    labels.append(label)
    return data, labels

images, labels = load_images_from_subfolders(base_path)
print(f"Loaded {len(images)} images.")

Loaded 26128 images.


In [1]:
images = np.array(images)
labels = np.array(labels)

print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

NameError: name 'np' is not defined

In [43]:
labels

array(['gallina', 'gallina', 'gallina', ..., 'elefante', 'elefante',
       'elefante'], dtype='<U10')

In [36]:
images = images.astype('float32') / 255.0
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [38]:
encoder = LabelBinarizer()
train_labels = encoder.fit_transform(train_labels)
test_labels = encoder.transform(test_labels)

In [40]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

datagen.fit(train_images)

In [45]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [46]:
for layer in base_model.layers:
    layer.trainable = False

In [48]:
model = models.Sequential()
model.add(base_model)  
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(len(encoder.classes_), activation='softmax'))

In [49]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [51]:
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) // 32,
                    epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 684s 1s/step - accuracy: 0.1895 - loss: 2.2071 - val_accuracy: 0.1808 - val_loss: 2.2080
Epoch 2/10
  1/653 ━━━━━━━━━━━━━━━━━━━━ 9:00 830ms/step - accuracy: 0.2812 - loss: 2.1192

2024-03-09 17:31:12.462749: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


653/653 ━━━━━━━━━━━━━━━━━━━━ 137s 209ms/step - accuracy: 0.2812 - loss: 1.0612 - val_accuracy: 0.1808 - val_loss: 2.2080
Epoch 3/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 719s 1s/step - accuracy: 0.1876 - loss: 2.2043 - val_accuracy: 0.1808 - val_loss: 2.2083
Epoch 4/10
  1/653 ━━━━━━━━━━━━━━━━━━━━ 9:38 887ms/step - accuracy: 0.2500 - loss: 2.1197

2024-03-09 17:45:28.435107: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


653/653 ━━━━━━━━━━━━━━━━━━━━ 135s 206ms/step - accuracy: 0.2500 - loss: 1.0615 - val_accuracy: 0.1808 - val_loss: 2.2083
Epoch 5/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 674s 1s/step - accuracy: 0.1880 - loss: 2.2084 - val_accuracy: 0.1808 - val_loss: 2.2082
Epoch 6/10
  1/653 ━━━━━━━━━━━━━━━━━━━━ 8:41 800ms/step - accuracy: 0.1875 - loss: 2.3456

...

653/653 ━━━━━━━━━━━━━━━━━━━━ 142s 216ms/step - accuracy: 0.1875 - loss: 1.1746 - val_accuracy: 0.1808 - val_loss: 2.2082
Epoch 7/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 690s 1s/step - accuracy: 0.1851 - loss: 2.2045 - val_accuracy: 0.1808 - val_loss: 2.2082
Epoch 8/10
  1/653 ━━━━━━━━━━━━━━━━━━━━ 8:33 788ms/step - accuracy: 0.2500 - loss: 2.2279

2024-03-09 18:12:48.674649: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


653/653 ━━━━━━━━━━━━━━━━━━━━ 139s 212ms/step - accuracy: 0.2500 - loss: 1.1156 - val_accuracy: 0.1808 - val_loss: 2.2082
Epoch 9/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 685s 1s/step - accuracy: 0.1920 - loss: 2.2050 - val_accuracy: 0.1808 - val_loss: 2.2082
Epoch 10/10
  1/653 ━━━━━━━━━━━━━━━━━━━━ 8:40 799ms/step - accuracy: 0.1875 - loss: 2.2949

2024-03-09 18:26:32.311328: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


653/653 ━━━━━━━━━━━━━━━━━━━━ 141s 215ms/step - accuracy: 0.1875 - loss: 1.1492 - val_accuracy: 0.1808 - val_loss: 2.2082


In [53]:
test_predictions = model.predict(test_images)
test_predictions = np.argmax(test_predictions, axis=1)
test_true_labels = np.argmax(test_labels, axis=1)

print(classification_report(test_true_labels, test_predictions, target_names=encoder.classes_))
print("Accuracy:", accuracy_score(test_true_labels, test_predictions))

164/164 ━━━━━━━━━━━━━━━━━━━━ 145s 879ms/step
              precision    recall  f1-score   support

        cane       0.18      1.00      0.31       945
     cavallo       0.00      0.00      0.00       506
    elefante       0.00      0.00      0.00       272
    farfalla       0.00      0.00      0.00       428
     gallina       0.00      0.00      0.00       612
       gatto       0.00      0.00      0.00       343
       mucca       0.00      0.00      0.00       381
      pecora       0.00      0.00      0.00       389
       ragno       0.00      0.00      0.00       988
  scoiattolo       0.00      0.00      0.00       362

    accuracy                           0.18      5226
   macro avg       0.02      0.10      0.03      5226
weighted avg       0.03      0.18      0.06      5226

Accuracy: 0.18082663605051666


/Users/aryansaxena/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aryansaxena/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aryansaxena/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu